In [1]:
import random
import utils
import numpy as np
import matplotlib.pyplot as plt

stack_size=10

# LeducGame class definition

In [2]:
class LeducGame:
    deck = []
    actions = [0,1,2] #0 is fold, 1 is check, 2 push
    firstplayer=None; #0 if player1 and 1 if player2
    hand_player1=0;
    hand_player2=0;
    boardcard=0;
    result=0;
    step_number=0
    roundGame=0 #0 preflop, 1 postflop
    game_round=0
    pot=None
    stack1=None
    stack2=None
    lastAction=None
    current_player=None
    game_is_over=None #0 game is not over, #1 game is over
    actions_hist=[]
    #inititate a game
    def __init__(self):
        self.deck = [0,0,1,1,2,2]
        
        #deal card to game from deck
        self.hand_player1=utils.choose_and_remove(self.deck)
        self.hand_player2=utils.choose_and_remove(self.deck)
        self.boardcard=utils.choose_and_remove(self.deck)
        self.result=self.get_result()
        self.firstplayer=random.randrange(0,2)
        #self.firstplayer=0
        self.step_number=0
        self.roundGame=0
        self.game_round=0
        self.pot=0
        self.stack1=stack_size
        self.stack2=stack_size
        self.current_player=self.firstplayer
        self.game_is_over=0 #0 not over, 1 over
        
        self.pot=2
        self.stack1=self.stack1-1
        self.stack2=self.stack2-1
        
    #allow to print leduc game state
    def __str__(self):
        return "FirstPlayer = {} \nHand1 = {} \nHand2 = {} \nBoard = {} \nDeck = {}\nResult = {}\nStack1={}\nStack2={}\nPot={}\nStep={}\nRound={}\nGameIsOver={}\nCurrent_player{}\n\n".format(self.firstplayer,self.hand_player1,self.hand_player2,self.boardcard,self.deck, self.result,self.stack1,self.stack2,self.pot,self.step_number,self.game_round,self.game_is_over, self.current_player)
     
    def get_firstplayer(self):
        return self.firstplayer
        
    def get_hand_player1(self):
        return self.hand_player1
    
    def get_hand_player2(self):
        return self.hand_player2
        
    def get_boardcard(self):
        return self.boardcard
        
    def get_current_player(self):
        return self.current_player
        
    def is_game_over(self):
        return self.game_is_over
    
    def get_game_round(self):
        return self.game_round
        
    #result() : 
    # 0  -> draw
    # 1  -> player1 win
    #-1  -> player2 win
    def get_result(self): #determine the best hand
        #Pairs
        if (self.hand_player1==self.boardcard):
            result=1
        elif (self.hand_player2==self.boardcard):
            result=-1
        #Highest card
        elif (self.hand_player1>self.hand_player2):
            result=1
        elif(self.hand_player1<self.hand_player2):
            result=-1
        #Draw
        else:
            result=0
        return result    
    
    def get_allowed_actions(self, action):
        if (action==0):
            return None
        elif (action==1):
            return [0,1,2]
        elif (action==2):
            return [0, 2]
        
    def step_prime(self, action):
        old_round=self.game_round
        gain=0
        
        #QAGENT
        if(self.current_player==0):
            if(action==0):
                gain=-1
                self.game_is_over=1
            elif(action==2):
                self.stack1=0

            #step1
            if(self.step_number==0):
                if(action==1):
                    self.lastAction=1
    
                if(action==2):
                    self.pot=12
                    self.lastAction=2
                self.step_number=1
            #step2        
            elif(self.step_number==1):
                if(action==1):
                    if(self.lastAction==1):
                        if(self.game_round==1):
                            self.game_is_over=1
                            
                            if(self.get_result()==1):
                                gain=1
                            if(self.get_result()==-1):
                                gain=-1
                            
                        self.game_round=1
                        self.step_number=0
                if(action==2):
                    if(self.lastAction==2):
                        self.pot=20
                        self.game_is_over=1
                        
                        if(self.get_result()==1):
                            gain=10
                        if(self.get_result()==-1):
                            gain=-10
                
                            
                    if(self.lastAction==1):
                        self.pot=12
                        self.step_number=2

            #step3
            elif(self.step_number==2):
                if(action==2):
                    self.pot=20
                    self.game_is_over=1
                    
                    if(self.get_result()==1):
                        gain=10
                    if(self.get_result()==-1):
                        gain=-10
                        
        #OPPONENT          
        elif(self.current_player==1):
            if(action==0):
                gain=1
                self.game_is_over=1
            elif(action==2):
                self.stack2=0

            #step1
            if(self.step_number==0):
                if(action==1):
                    self.lastAction=1
                if(action==2):
                    self.pot=12
                    self.lastAction=2
                self.step_number=1
                    
            #step2        
            elif(self.step_number==1):
                if(action==1):
                    if(self.lastAction==1):
                        if(self.game_round==1):
                            self.game_is_over=1
                            
                            if(self.get_result()==1):
                                gain=1
                            if(self.get_result()==-1):
                                gain=-1
                            
                        self.game_round=1
                        self.step_number=0
                if(action==2):
                    if(self.lastAction==2):
                        self.pot=20
                        self.game_is_over=1
                        
                        if(self.get_result()==1):
                            gain=10
                        if(self.get_result()==-1):
                            gain=-10
                            
                    if(self.lastAction==1):
                        self.pot=12
                        self.step_number=2
            #step3
            elif(self.step_number==2):
                if(action==2):
                    self.pot=20
                    self.game_is_over=1
                    
                    if(self.get_result()==1):
                        gain=10
                    if(self.get_result()==-1):
                        gain=-10
            
        allowed_actions= self.get_allowed_actions(action)
        #print("action= ", action)
        #print(self)
        if(old_round==self.game_round):
            if(self.current_player==0):
                self.current_player=1
            elif(self.current_player==1):
                self.current_player=0
            #self.current_player=not(self.current_player)
        else:
            self.current_player=self.firstplayer   
        
        return gain,self.current_player, allowed_actions          

# Environment

In [3]:
class Environment:
    agent=None
    game=None
    opponent_action=None
    actions_hist=[]
    
    def __init__(self,agent):
        self.game=LeducGame()
        self.agent=agent
        
    def __str__(self):
        return "{} \nOpponent_Action = {}\nStack = {}".format(self.game,self.agent.get_action(),self.stack)
    
    def get_actions_hist(self):
        return self.actions_hist
    
    def set_opponent_action(self, action):
        self.opponent_action=action
    
    def get_state(self):
        hand1=self.game.get_hand_player1()
        first=self.game.get_firstplayer()
        boardcard= self.game.get_boardcard()
        
        if self.game.game_round == 0:
            if first is 0:
                return hand1
            elif(first==1 and self.opponent_action==1):
                return hand1+3
            elif(first==1 and self.opponent_action==2):
                return hand1+6
            
        elif self.game.game_round == 1:
            if first is 0:
                if hand1==0:
                    return boardcard+9
                elif hand1== 1:
                    return boardcard+12
                elif hand1== 2:
                    return boardcard+15
            else:
                if (self.opponent_action==1):
                    if hand1==0:
                        return boardcard+18
                    elif hand1==1:
                        return boardcard+21
                    elif hand1==2:
                        return boardcard+24
                if(self.opponent_action==2):
                    if hand1==0:
                        return boardcard+27
                    elif hand1==1:
                        return boardcard+30
                    elif hand1==2:
                        return boardcard+33
    
    def reset(self):
        self.game=LeducGame()
        self.actions_hist=[]
    
    def step(self, qagent_action):
        r=0
        state=None
        
        self.actions_hist.append([self.get_state(),qagent_action])
        r,current_player,allowed_actions=self.game.step_prime(qagent_action)
        if(allowed_actions==None):
            self.game.game_is_over=1
        if(self.game.is_game_over()==0):
            self.agent.set_action(allowed_actions,qagent_action, self.game.get_game_round(), self.game.get_hand_player2(),self.game.get_boardcard())
            self.opponent_action=self.agent.get_action()
            #print( "agent action", self.opponent_action)
            r,current_player,allowed_actions= self.game.step_prime(self.opponent_action)
            if(self.game.is_game_over()==1):
                state=None
            else:
                state=self.get_state()
        
        return r, allowed_actions, state
            

## Random Agent

In [4]:
class RandAgent():
    
    action=None
          
        
    def __init__(self):
        #qtable creation
        self.set_action([0,1,2], 0,0,0,0)
        
    def set_action(self,allowed_actions,qagent_action, game_round, hand, boardcard):            
        self.action=random.choice(allowed_actions)
        
    def get_action(self):
        return self.action

## Greedy Agent

In [5]:
class GreedyAgent():
    
    action=None
        
    def set_action(self,allowed_actions,qagent_action, game_round, hand, boardcard):            
        if game_round == 0:
            if qagent_action == None:
                if hand == 0:
                    self.action= 1
                elif hand == 1:
                    self.action= 1
                elif hand == 2:
                    self.action= 1
            elif qagent_action == 1:
                if hand == 0:
                    self.action= 1
                elif hand == 1:
                    self.action= 1
                elif hand == 2:
                    self.action= 1
            elif qagent_action == 2:
                if hand == 0:
                    self.action= 0
                elif hand == 1:
                    self.action= 0
                elif hand == 2:
                    self.action= 2
        elif game_round == 1:
            if qagent_action == None:
                if hand == 0:
                    if boardcard == 0:
                        self.action= 2
                    elif boardcard == 1:
                        self.action= 0
                    elif boardcard == 2:
                        self.action= 0
                elif hand == 1:
                    if boardcard == 0:
                        self.action= 1
                    elif boardcard == 1:
                        self.action= 2
                    elif boardcard == 2:
                        self.action= 0
                elif hand == 2:
                    if boardcard == 0:
                        self.action= 1
                    elif boardcard == 1:
                        self.action= 1
                    elif boardcard == 2:
                        self.action= 2
            elif qagent_action == 1:
                if hand == 0:
                    if boardcard == 0:
                        self.action= 2
                    elif boardcard == 1:
                        self.action= 0
                    elif boardcard == 2:
                        self.action= 0
                elif hand == 1:
                    if boardcard == 0:
                        self.action= 1
                    elif boardcard == 1:
                        self.action= 2
                    elif boardcard == 2:
                        self.action= 1
                elif hand == 2:
                    if boardcard == 0:
                        self.action= 2
                    elif boardcard == 1:
                        self.action= 2
                    elif boardcard == 2:
                        self.action= 2
            elif qagent_action == 2:
                if hand == 0:
                    if boardcard == 0:
                        self.action= 2
                    elif boardcard == 1:
                        self.action= 0
                    elif boardcard == 2:
                        self.action= 0
                elif hand == 1:
                    if boardcard == 0:
                        self.action= 0
                    elif boardcard == 1:
                        self.action= 2
                    elif boardcard == 2:
                        self.action= 0
                elif hand == 2:
                    if boardcard == 0:
                        self.action= 0
                    elif boardcard == 1:
                        self.action= 0
                    elif boardcard == 2:
                        self.action= 2
        
    def get_action(self):
        return self.action

## Human Agent

In [6]:
class HumanAgent():
    
    action=None
        
    def set_action(self,allowed_actions,qagent_action, game_round, hand, boardcard):            
        if (game_round==0):
            if (qagent_action==None):
                print("Your card = ",hand,"\n\n")
            else:
                print("Your card =", hand ,"\nAction of the QAgent = ",qagent_action,"\n\n")
        elif (game_round==1):
            if (qagent_action==None):
                print("Your card =", hand ,"\nBoardcard = ",boardcard ,"\n\n")
            else:
                print("Your card =", hand, "\nAction of the QAgent = ",qagent_action,"\nBoardcard = ", boardcard," \n\n")
        self.action = input("Please enter an action :(0,1 or 2)\n")
        
        
    def get_action(self):
        return self.action

## QAgent

In [7]:
class QAgent():
    qtable=[]
 # Agent class definition   state=None
    learning_rate=0.1
    
    state=None #0-5
    gamma=1
    state_number=36
    state=None 
    perf=[]
    
    def __init__(self):
        #qtable creation
        self.qtable=np.zeros((self.state_number,3))
        self.perf=[]
        
    #allow to print leduc game state
    def __str__(self):
        return "State = {} \nQTable = {} \nLearning rate = {}".format(self.state,self.qtable,self.learning_rate)
    
    def explore_action(self, allowed_actions):
        action=random.choice(allowed_actions)
        return action
    
    def exploit_action(self, allowed_actions):
        action=utils.get_max_list(self.qtable[self.state], allowed_actions)
        return action
      
    def set_state(self, state):
        self.state=state    
        
    def set_qtable(self,qtable):
        self.qtable=qtable
        
    def set_perf(self,perf):
        self.perf.append(perf)

    def update(self,reward, actions_hist):
        #print("UPDATE action= {} \n reward = {} \n next_state = {}\n".format(action,reward,next_state))
        new_value = 0
        
        #print ("Update: \n")
        #print ("reward = ", reward)
        #print ("actions_hist = ", actions_hist)
        if(len(actions_hist)==1):
            new_value = (1 - self.learning_rate) * self.qtable[actions_hist[0][0], actions_hist[0][1]] +  self.learning_rate * reward
            self.qtable[actions_hist[0][0], actions_hist[0][1]] = new_value
            #print(actions_hist)
        if(len(actions_hist)==2):
            new_value = (1 - self.learning_rate) * self.qtable[actions_hist[1][0], actions_hist[1][1]] +  self.learning_rate * reward
            self.qtable[actions_hist[1][0], actions_hist[1][1]] = new_value

            back = (1 - self.learning_rate) * self.qtable[actions_hist[0][0], actions_hist[0][1]] +  self.learning_rate * (0 + self.gamma * reward)
            self.qtable[actions_hist[0][0], actions_hist[0][1]] = back

        if(len(actions_hist)==3):
            new_value = (1 - self.learning_rate) * self.qtable[actions_hist[2][0], actions_hist[2][1]] +  self.learning_rate * reward
            self.qtable[actions_hist[2][0], actions_hist[2][1]] = new_value

            back = (1 - self.learning_rate) * self.qtable[actions_hist[1][0], actions_hist[1][1]] +  self.learning_rate * (0 + self.gamma * reward)
            self.qtable[actions_hist[1][0], actions_hist[1][1]] = back

            back_prime = (1 - self.learning_rate) * self.qtable[actions_hist[0][0], actions_hist[0][1]] +  self.learning_rate * (0 + self.gamma * reward)
            self.qtable[actions_hist[0][0], actions_hist[0][1]] = back_prime

# Training and Testing parameters

In [8]:
epochs_number=100000
evaluate_every=30
test_number=1000

## qAgent vs RandAgent

In [9]:
randAgent=RandAgent()
env=Environment(randAgent)
qagent=QAgent()

for i in range(epochs_number):
    allowed_actions=[0,1,2]
    current_player=env.game.get_firstplayer()
    while(env.game.is_game_over()==0):
        state=env.get_state()
        qagent.set_state(state)
        
        if (current_player==1):
            env.agent.set_action(allowed_actions,0,0,0,0)
            env.set_opponent_action(env.agent.get_action())
            reward,current_player,allowed_actions=env.game.step_prime(env.agent.get_action())
        if(env.game.is_game_over()==0):
            qagent_action=qagent.explore_action(allowed_actions)
            reward,allowed_actions, new_state=env.step(qagent_action)
            if(env.game.is_game_over()==1):
                qagent.update(reward, env.get_actions_hist())
            qagent.set_state(new_state)
        else:
            env.reset()
            allowed_actions=[0,1,2]
            first_time=True
    
    env.reset()
    
with np.printoptions(precision=3, suppress=True):
    print(qagent.qtable)

[[-0.643  0.051  1.08 ]
 [-0.576 -0.096  0.758]
 [-0.327  0.062  2.132]
 [-1.    -0.94  -0.441]
 [-1.     0.341  1.703]
 [-1.     1.705  4.063]
 [-0.9    0.    -4.872]
 [-0.695  0.     0.03 ]
 [-0.8    0.     6.059]
 [-1.     1.722  7.704]
 [-1.    -0.114 -4.328]
 [-1.    -1.133 -3.13 ]
 [-1.    -1.776 -1.454]
 [-1.     2.758  5.215]
 [-1.     0.123  0.849]
 [-1.    -0.54  -0.603]
 [-1.    -0.091  0.197]
 [-1.     2.948  7.236]
 [-0.24   0.997  5.444]
 [ 0.03  -0.758 -2.655]
 [-0.676 -2.352 -2.952]
 [-0.244 -0.812 -4.482]
 [-0.478  1.496  5.751]
 [-0.197 -0.893  0.771]
 [-0.573  0.101  2.887]
 [-0.582 -0.163  1.792]
 [-0.519  3.138  4.047]
 [-0.902  0.     9.992]
 [-0.877  0.    -7.475]
 [-0.998  0.    -9.14 ]
 [-0.995  0.    -8.804]
 [-0.995  0.     9.984]
 [-0.837  0.     0.197]
 [-0.859  0.     0.26 ]
 [-0.975  0.     1.964]
 [-0.949  0.     9.984]]


## qAgent vs GreedyAgent

In [10]:
greedyAgent=GreedyAgent()
env=Environment(greedyAgent)
qagent=QAgent()

for i in range(epochs_number):
    allowed_actions=[0,1,2]
    current_player=env.game.get_firstplayer()
    while(env.game.is_game_over()==0):
        state=env.get_state()
        qagent.set_state(state)
        
        if (current_player==1):
            env.agent.set_action(allowed_actions,qagent_action, env.game.get_game_round(), env.game.get_hand_player2(),env.game.get_boardcard())

            env.set_opponent_action(env.agent.get_action())
            reward,current_player,allowed_actions=env.game.step_prime(env.agent.get_action())
        if(env.game.is_game_over()==0):
            qagent_action=qagent.explore_action(allowed_actions)
            reward,allowed_actions, new_state=env.step(qagent_action)
            if(env.game.is_game_over()==1):
                qagent.update(reward, env.get_actions_hist())
            qagent.set_state(new_state)
        else:
            env.reset()
            allowed_actions=[0,1,2]
            first_time=True
    
    env.reset()
    
with np.printoptions(precision=3, suppress=True):
    print(qagent.qtable)

[[  0.242  -0.332   0.484]
 [ -1.864  -3.313  -2.276]
 [ -2.602  -2.016  -1.09 ]
 [ -1.     -2.69   -2.737]
 [ -1.     -0.968   1.252]
 [ -1.     -0.305   0.773]
 [ -1.      0.     -4.823]
 [ -1.      0.     -3.687]
 [ -1.      0.     -0.631]
 [ -1.      3.177   2.393]
 [ -1.     -3.692  -4.659]
 [ -1.     -2.07   -0.97 ]
 [ -1.     -4.396  -4.952]
 [ -1.      2.697   3.535]
 [ -1.     -0.688  -1.763]
 [ -1.     -0.01   -1.429]
 [ -1.     -2.271  -1.185]
 [ -1.      1.      1.   ]
 [  0.586   1.501  -0.8  ]
 [  1.     -4.735  -3.018]
 [  0.04   -0.909  -1.762]
 [  0.428  -3.087  -2.371]
 [  1.      3.287  -1.   ]
 [  0.659  -0.665  -2.182]
 [  0.519  -0.194  -1.381]
 [  1.     -0.981  -3.034]
 [  0.203   0.058  -0.367]
 [ -0.478   0.      9.987]
 [ -0.073   0.    -10.   ]
 [ -0.014   0.     -9.997]
 [ -0.725   0.    -10.   ]
 [ -0.349   0.      9.997]
 [ -0.267   0.     -9.996]
 [ -0.262   0.     -5.524]
 [ -0.538   0.     -2.653]
 [  0.      0.      0.   ]]


## qAgent vs HumanAgent

In [15]:
humanAgent=HumanAgent()
env=Environment(humanAgent)
qagent=QAgent()

for i in range(epochs_number):
    allowed_actions=[0,1,2]
    current_player=env.game.get_firstplayer()
    while(env.game.is_game_over()==0):
        state=env.get_state()
        qagent.set_state(state)
        
        if (current_player==1):
            print("coucou")
            env.agent.set_action(allowed_actions,None, env.game.get_game_round(), env.game.get_hand_player2(),env.game.get_boardcard())
            print("human action", env.agent.get_action())
            env.set_opponent_action(env.agent.get_action())
            reward,current_player,allowed_actions=env.game.step_prime(env.agent.get_action())
        if(env.game.is_game_over()==0):
            print("allowed_actions", allowed_actions)
            qagent_action=qagent.explore_action(allowed_actions)
            reward,allowed_actions, new_state=env.step(qagent_action)
            if(env.game.is_game_over()==1):
                qagent.update(reward, env.get_actions_hist())
            qagent.set_state(new_state)
        else:
            env.reset()
            allowed_actions=[0,1,2]
            first_time=True
    
    env.reset()
    
with np.printoptions(precision=3, suppress=True):
    print(qagent.qtable)

coucou
Your card =  1 


Please enter an action :(0,1 or 2)
1
human action 1
allowed_actions None


TypeError: object of type 'NoneType' has no len()

In [ ]:
qagent.qtable[0][2]

In [ ]:
game=LeducGame()
game.step_prime(1)
print(game)
game.step_prime(2)
print(game)
game.step_prime(2)
print(game)